1.  Preparação dos Dados

b)Realização de limpeza necessária (tratar valores nulos, remover duplicatas, etc.)

In [ ]:


import os
import pandas as pd

# Pasta para salvar os dados limpos
pasta_saida = "data_limpa"
os.makedirs(pasta_saida, exist_ok=True)

# Caminho para a pasta onde estão os arquivos CSV
pasta_dados = 'c:/Users/Marisa/Desktop/trigoo-olist-project/dados'

for nome, tabela in tabelas.items():
    df = tabela.copy()
    n_antes = len(df)

    # Remover linhas com valores nulos
    df = df.dropna()

    # Remover duplicatas
    df = df.drop_duplicates()

    # Padronização de tipos
    for coluna in df.columns:
        col_lower = coluna.lower()
        # Converter datas
        if "date" in col_lower or "timestamp" in col_lower:
            try:
                df[coluna] = pd.to_datetime(df[coluna], errors="coerce")
            except Exception:
                pass
        # Converter ids e prefixos postais para string
        elif ("id" in col_lower or "zip_code_prefix" in col_lower or "code" in col_lower):
            df[coluna] = df[coluna].astype(str)

    n_depois = len(df)
    print(f"{nome}: removidas {n_antes - n_depois} linhas com nulos/duplicatas e tipos padronizados.")

    # Salvar arquivo CSV
    caminho_arquivo = os.path.join(pasta_saida, f"{nome}_limpa.csv")
    df.to_csv(caminho_arquivo, index=False)
    print(f"Salvo: {caminho_arquivo}")


olist_customers_dataset: removidas 0 linhas com nulos/duplicatas e tipos padronizados.
Salvo: data_limpa\olist_customers_dataset_limpa.csv
olist_geolocation_dataset: removidas 261831 linhas com nulos/duplicatas e tipos padronizados.
Salvo: data_limpa\olist_geolocation_dataset_limpa.csv
olist_orders_dataset: removidas 2980 linhas com nulos/duplicatas e tipos padronizados.
Salvo: data_limpa\olist_orders_dataset_limpa.csv
olist_order_items_dataset: removidas 0 linhas com nulos/duplicatas e tipos padronizados.
Salvo: data_limpa\olist_order_items_dataset_limpa.csv
olist_order_payments_dataset: removidas 0 linhas com nulos/duplicatas e tipos padronizados.
Salvo: data_limpa\olist_order_payments_dataset_limpa.csv
olist_order_reviews_dataset: removidas 89385 linhas com nulos/duplicatas e tipos padronizados.
Salvo: data_limpa\olist_order_reviews_dataset_limpa.csv
olist_products_dataset: removidas 611 linhas com nulos/duplicatas e tipos padronizados.
Salvo: data_limpa\olist_products_dataset_limpa

c) Identificando as colunas presentes nas tabelas tratadas para fazer a relação entre elas

In [7]:
import os
import pandas as pd

pasta_limpa = "data_limpa"
arquivos = [f for f in os.listdir(pasta_limpa) if f.endswith(".csv")]

for arquivo in arquivos:
    caminho = os.path.join(pasta_limpa, arquivo)
    df = pd.read_csv(caminho)
    print(f"📄 Tabela: {arquivo.replace('_limpa.csv', '')}")
    print(f" - Colunas ({len(df.columns)}): {list(df.columns)}\n")


📄 Tabela: olist_customers_dataset
 - Colunas (5): ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']

📄 Tabela: olist_geolocation_dataset
 - Colunas (5): ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']

📄 Tabela: olist_orders_dataset
 - Colunas (8): ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

📄 Tabela: olist_order_items_dataset
 - Colunas (7): ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

📄 Tabela: olist_order_payments_dataset
 - Colunas (5): ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']

📄 Tabela: olist_order_reviews_dataset
 - Colunas (7): ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_me

d) Normalização das tabelas e criação de um modelo relacional

In [8]:
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey, UniqueConstraint
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

Base = declarative_base()

# Tabelas lookup e principais (igual antes)
class ProductCategory(Base):
    __tablename__ = "product_category"
    category_id = Column(Integer, primary_key=True, autoincrement=True)
    product_category_name = Column(String, unique=True, nullable=False)
    product_category_name_english = Column(String)

class OrderStatus(Base):
    __tablename__ = "order_status"
    status_id = Column(Integer, primary_key=True, autoincrement=True)
    status_name = Column(String, unique=True, nullable=False)

class PaymentType(Base):
    __tablename__ = "payment_type"
    payment_type_id = Column(Integer, primary_key=True, autoincrement=True)
    payment_type_name = Column(String, unique=True, nullable=False)

class Location(Base):
    __tablename__ = "location"
    location_id = Column(Integer, primary_key=True, autoincrement=True)
    zip_code_prefix = Column(Integer, nullable=False)
    city = Column(String, nullable=False)
    state = Column(String, nullable=False)
    __table_args__ = (UniqueConstraint('zip_code_prefix', 'city', 'state', name='_zip_city_state_uc'),)

class Customer(Base):
    __tablename__ = "olist_customers_dataset"
    customer_id = Column(String, primary_key=True)
    customer_unique_id = Column(String, nullable=False)
    location_id = Column(Integer, ForeignKey('location.location_id'))
    location = relationship("Location")

class Seller(Base):
    __tablename__ = "olist_sellers_dataset"
    seller_id = Column(String, primary_key=True)
    location_id = Column(Integer, ForeignKey('location.location_id'))
    location = relationship("Location")

class Product(Base):
    __tablename__ = "olist_products_dataset"
    product_id = Column(String, primary_key=True)
    category_id = Column(Integer, ForeignKey('product_category.category_id'))
    product_name_lenght = Column(Integer)
    product_description_lenght = Column(Integer)
    product_photos_qty = Column(Integer)
    product_weight_g = Column(Float)
    product_length_cm = Column(Float)
    product_height_cm = Column(Float)
    product_width_cm = Column(Float)
    category = relationship("ProductCategory")

class Order(Base):
    __tablename__ = "olist_orders_dataset"
    order_id = Column(String, primary_key=True)
    customer_id = Column(String, ForeignKey('olist_customers_dataset.customer_id'))
    status_id = Column(Integer, ForeignKey('order_status.status_id'))
    order_purchase_timestamp = Column(DateTime)
    order_approved_at = Column(DateTime)
    order_delivered_carrier_date = Column(DateTime)
    order_delivered_customer_date = Column(DateTime)
    order_estimated_delivery_date = Column(DateTime)

    customer = relationship("Customer")
    status = relationship("OrderStatus")

class OrderItem(Base):
    __tablename__ = "olist_order_items_dataset"
    order_id = Column(String, ForeignKey('olist_orders_dataset.order_id'), primary_key=True)
    order_item_id = Column(Integer, primary_key=True)
    product_id = Column(String, ForeignKey('olist_products_dataset.product_id'))
    seller_id = Column(String, ForeignKey('olist_sellers_dataset.seller_id'))
    shipping_limit_date = Column(DateTime)
    price = Column(Float)
    freight_value = Column(Float)

    order = relationship("Order")
    product = relationship("Product")
    seller = relationship("Seller")

class OrderPayment(Base):
    __tablename__ = "olist_order_payments_dataset"
    order_id = Column(String, ForeignKey('olist_orders_dataset.order_id'), primary_key=True)
    payment_sequential = Column(Integer, primary_key=True)
    payment_type_id = Column(Integer, ForeignKey('payment_type.payment_type_id'))
    payment_installments = Column(Integer)
    payment_value = Column(Float)

    order = relationship("Order")
    payment_type = relationship("PaymentType")

class OrderReview(Base):
    __tablename__ = "olist_order_reviews_dataset"
    review_id = Column(String, primary_key=True)
    order_id = Column(String, ForeignKey('olist_orders_dataset.order_id'))
    review_score = Column(Integer)
    review_comment_title = Column(String)
    review_comment_message = Column(String)
    review_creation_date = Column(DateTime)
    review_answer_timestamp = Column(DateTime)

    order = relationship("Order")

# Criar engine em memória (não salva em disco)
engine = create_engine("sqlite:///:memory:")

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

print("Modelo relacional normalizado criado em memória no notebook")


Modelo relacional normalizado criado em memória no notebook


## 🧩 Modelo Relacional Normalizado – Olist

Este modelo foi criado com base nos datasets da Olist, realizando:

- **Remoção de nulos e duplicatas**
- **Padronização dos tipos de dados**
- **Normalização das tabelas para evitar redundância**
- **Relacionamentos com chaves estrangeiras entre as tabelas**

### 🔗 Principais Tabelas e Relacionamentos

- **Customer (`olist_customers_dataset`)**
  - Relacionado com `Location` via `location_id`

- **Seller (`olist_sellers_dataset`)**
  - Relacionado com `Location` via `location_id`

- **Order (`olist_orders_dataset`)**
  - Relacionado com `Customer`
  - Relacionado com `OrderStatus`

- **OrderItem (`olist_order_items_dataset`)**
  - Relacionado com `Order`, `Product`, `Seller`

- **Product (`olist_products_dataset`)**
  - Relacionado com `ProductCategory`

- **OrderPayment (`olist_order_payments_dataset`)**
  - Relacionado com `Order`
  - Relacionado com `PaymentType`

- **OrderReview (`olist_order_reviews_dataset`)**
  - Relacionado com `Order`

- **Location**
  - Tabela de apoio (zip_code_prefix, cidade, estado)

- **ProductCategory**
  - Inclui nome original e nome em inglês

- **OrderStatus** e **PaymentType**
  - Tabelas de domínio para facilitar padronização

### 🛠️ Observações Técnicas

- O modelo foi implementado em SQLite diretamente no notebook para facilitar testes rápidos.
- As tabelas foram normalizadas para reduzir redundâncias e facilitar manutenções e análises futuras.
- Caso deseje persistir o banco, basta alterar a engine para salvar em disco:  
  ```python
  engine = create_engine("sqlite:///olist_relacional.db")
